In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd

'c:\\Users\\PC\\Documents\\projects\\Power_Consumption_In_Tetouan'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    features_data_path: Path
    targets_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [5]:
from powerconsumptiontetouan.constants import *
from powerconsumptiontetouan.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            features_data_path = config.features_data_path,
            targets_data_path = config.targets_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
            
        )

        return model_trainer_config

In [7]:
import pandas as pd
import os
from powerconsumptiontetouan import logger
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
import joblib

In [8]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        features_data = pd.read_csv(self.config.features_data_path)
        targets_data = pd.read_csv(self.config.targets_data_path)

        # zone_1 = targets_data.columns[0]
        # zone_2 = targets_data.columns[1]
        # zone_3 = targets_data.columns[2]

        X_train, X_test, y_train, y_test = train_test_split(features_data, targets_data[self.config.target_column],
                                                                                                   test_size = 0.30,
                                                                                                   random_state = 0)

        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(X_train, y_train)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))



In [9]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-07-28 09:29:15,654: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-28 09:29:15,655: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-28 09:29:15,656: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-28 09:29:15,657: INFO: common: created directory at: artifacts]
[2024-07-28 09:29:15,658: INFO: common: created directory at: artifacts/model_training]
